In [10]:
from hammers.osapi import Auth, load_osrc
from hammers.osrest import glance, keystone

In [5]:
rc = {
    'tacc': load_osrc('../rc/admin-chi.tacc.rc', get_pass=True),
    'uc': load_osrc('../rc/admin-chi.uc.rc'),
    'kvm': load_osrc('../rc/admin-kvm.tacc.rc'),
}
rc['uc']['OS_PASSWORD'] = rc['kvm']['OS_PASSWORD'] = rc['tacc']['OS_PASSWORD']
auth = {k: Auth(rc[k]) for k in rc}

Enter your password: ········


In [15]:
glance.images(auth['tacc'], {'name': 'CC-CentOS7', 'visibility': 'public'})

[{'checksum': '0279648e547ce8b187f985eaeafeede2',
  'container_format': 'bare',
  'created_at': '2016-12-23T18:19:51Z',
  'disk_format': 'qcow2',
  'file': '/v2/images/0f216b1f-7841-451b-8971-d383364e01a6/file',
  'id': '0f216b1f-7841-451b-8971-d383364e01a6',
  'min_disk': 0,
  'min_ram': 0,
  'name': 'CC-CentOS7',
  'owner': '570aad8999f7499db99eae22fe9b29bb',
  'protected': False,
  'schema': '/v2/schemas/image',
  'self': '/v2/images/0f216b1f-7841-451b-8971-d383364e01a6',
  'size': 734445056,
  'status': 'active',
  'tags': [],
  'updated_at': '2016-12-23T19:14:23Z',
  'virtual_size': None,
  'visibility': 'public'}]

In [31]:
past_centos = {
    'tacc': {'id': '0f216b1f-7841-451b-8971-d383364e01a6'},
    'uc': {'id': '2c16bd4a-8d77-4c56-ae5d-fca651147016'},
    'kvm': {'id': 'dc59301a-8d0f-4125-a09f-8ae1965fda64'},
}
past_xenial = {
    'tacc': {'id': 'decc379b-1bf4-42b6-80a3-1a73836ef520'},
    'uc': {'id': 'a29be6ec-cf43-488d-9fb6-9a257c96b0f6'},
    'kvm': {'id': '2808416b-f1b6-4429-9a4f-a03a1b0da4c3'},
}
new_centos = {
    'tacc': {'id': '10adbb3b-ccd3-4e23-8623-b8df82073de9'},
    'uc': {'id': '388ca9a2-c7d7-49ac-8f66-6b254a6951d4'},
    'kvm': {'id': '053b3438-edb5-4d17-873f-525600b9d9a9'},
}
new_xenial = {
    'tacc': {'id': 'd930e8b3-7b29-4d25-9a63-cefdc46d6bc4'},
    'uc': {'id': '926d0ea3-6e47-40f3-87ba-596f3bbe56cc'},
    'kvm': {'id': 'c80567e8-12f9-44fc-8335-f62831ee73a6'},
}

In [25]:
for site, image in past_centos.items():
#     image['details'] = glance.image(auth[site], image['id'])
#     assert image['details']['name'] == 'CC-CentOS7'
    glance.image_properties(auth[site], image['id'], add={
        'build-os': 'centos7'
        'build-os-base-image-revision': '1612.0',
    })
    

In [30]:
for site, image in past_xenial.items():
    image['details'] = glance.image(auth[site], image['id'])
    assert image['details']['name'] == 'CC-Ubuntu16.04'
    glance.image_properties(auth[site], image['id'], add={
        'build-os': 'ubuntu-xenial',
        'build-os-base-image-revision': '20170410',
    })

In [28]:
for site in auth:
    print(site)
    images = glance.images(auth[site])

    xenial = max((i for i in images if i.get('build-os') == 'ubuntu-xenial'), key=lambda i: i['created_at'])
    centos = max((i for i in images if i.get('build-os') == 'centos7'), key=lambda i: i['created_at'])

    print(xenial['id'], xenial['name'])
    print(centos['id'], centos['name'])
    print()

tacc
d930e8b3-7b29-4d25-9a63-cefdc46d6bc4 rc-CC-Ubuntu16.04
10adbb3b-ccd3-4e23-8623-b8df82073de9 rc-CC-CentOS7

uc
926d0ea3-6e47-40f3-87ba-596f3bbe56cc rc-CC-Ubuntu16.04
388ca9a2-c7d7-49ac-8f66-6b254a6951d4 rc-CC-CentOS7

kvm
c80567e8-12f9-44fc-8335-f62831ee73a6 rc-CC-Ubuntu16.04
053b3438-edb5-4d17-873f-525600b9d9a9 rc-CC-CentOS7



In [56]:
iterprops = iter(set(i) for i in images)
init = next(iterprops)

always_props = set(init)
all_props = set(init)

for props in iterprops:
    always_props &= props
    all_props |= props
    
sometimes_props = all_props - always_props
sometimes_props

{'build-os',
 'build-os-base-image-revision',
 'build-repo',
 'build-repo-commit',
 'build-tag',
 'build-variant'}

In [57]:
def extract_extra_properties(image):
    return {k: image[k] for k in sometimes_props if k in image}

In [58]:
extract_extra_properties(centos)

{'build-os': 'centos7',
 'build-os-base-image-revision': '1710',
 'build-repo': 'https://github.com/ChameleonCloud/CC-CentOS7',
 'build-repo-commit': 'c6c97c59133a7648f67c333d1dd889182de9bc5a',
 'build-tag': 'jenkins-cc-centos7-builder-42',
 'build-variant': 'base'}

# Site-to-site Transport

In [59]:
import fabric.api as fapi
import fabric.context_managers as fcm
import fabric.network as fnet

In [60]:
fapi.env['use_ssh_config'] = True
fapi.env['abort_on_prompts'] = True
fapi.env['host_string'] = 'm01-07'

In [61]:
fnet.disconnect_all()

Disconnecting from root@m01-07.chameleon.tacc.utexas.edu... done.


In [71]:
SOURCE_IMAGE = centos # dict
SOURCE_SITE_AUTH = auth['tacc']
TARGET_SITE_AUTH = auth['uc']
TARGET_IMAGE_NAME = 'rc-CC-CentOS7'

_TEMP_FILE = 'temp.img'

curl_download = glance.image_download_curl(SOURCE_SITE_AUTH, SOURCE_IMAGE['id'], filepath=_TEMP_FILE)
with fcm.cd('npt'):
    fapi.run('rm -f {}'.format(_TEMP_FILE))
    out = fapi.run(curl_download, quiet=True)
    assert out.return_code == 0
    
    new_image = glance.image_create(TARGET_SITE_AUTH, TARGET_IMAGE_NAME, public=True, extra=extract_extra_properties(SOURCE_IMAGE))
    curl_upload = glance.image_upload_curl(TARGET_SITE_AUTH, new_image['id'], _TEMP_FILE)
    out = fapi.run(curl_upload, quiet=True)
    assert out.return_code == 0
    
new_image

[m01-07] run: rm -f temp.img


{'build-os': 'centos7',
 'build-os-base-image-revision': '1710',
 'build-repo': 'https://github.com/ChameleonCloud/CC-CentOS7',
 'build-repo-commit': 'c6c97c59133a7648f67c333d1dd889182de9bc5a',
 'build-tag': 'jenkins-cc-centos7-builder-42',
 'build-variant': 'base',
 'checksum': None,
 'container_format': 'bare',
 'created_at': '2017-11-06T23:27:14Z',
 'disk_format': 'qcow2',
 'file': '/v2/images/388ca9a2-c7d7-49ac-8f66-6b254a6951d4/file',
 'id': '388ca9a2-c7d7-49ac-8f66-6b254a6951d4',
 'min_disk': 0,
 'min_ram': 0,
 'name': 'rc-CC-CentOS7',
 'owner': 'cdf92cfdf4c5422cb2ead2d8a8ca2ae7',
 'protected': False,
 'schema': '/v2/schemas/image',
 'self': '/v2/images/388ca9a2-c7d7-49ac-8f66-6b254a6951d4',
 'size': None,
 'status': 'queued',
 'tags': [],
 'updated_at': '2017-11-06T23:27:14Z',
 'virtual_size': None,
 'visibility': 'private'}

In [ ]:
with fcm.cd('npt'):
    new_image = glance.image_create(auth['tacc'], 'testimage', extra=extract_extra_properties(xenial))
    curl_upload = glance.image_upload_curl(auth['tacc'], new_image['id'], 'temp.img')
    out = fapi.run(curl_upload, quiet=True)

In [52]:
new_image = glance.image(auth['tacc'], new_image['id'])
assert new_image['checksum'] == xenial['checksum']

# Swap Image Names

In [42]:
for site in auth:
    glance.image_properties(auth[site], past_xenial[site]['id'], replace={'name': 'CC-Ubuntu16.04-20170410'})
    glance.image_properties(auth[site], new_xenial[site]['id'], replace={'name': 'CC-Ubuntu16.04'})
    glance.image_properties(auth[site], past_centos[site]['id'], replace={'name': 'CC-CentOS7-1612.0'})
    glance.image_properties(auth[site], new_centos[site]['id'], replace={'name': 'CC-CentOS7'})